In [ ]:
# old code to place order according buy first and according to the market change the signals 

# Main trading logic
order_count = 0
pnl = 1  # Start with profit state
inPosition = False
buying_price = 0
selling_price = 0

# pnl = find_pos()

while order_count < 100:
    current_time = datetime.datetime.now().time()
    if current_time >= datetime.time(9, 0) and current_time <= datetime.time(15,0):
        if not inPosition:
            # the strategy is starts from here
            if pnl == 1:
                buy_order = place_order('B')
                res = client.order_report()
                ltp = float(redis_client.hget('stock_data', TOKEN))
                buying_price = float(res['data'][-1]['avgPrc'])
                
                if buy_order and buy_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Bought at {buying_price} , Ltp is : {ltp}")
                    
                    inPosition = True
                    tar = buying_price * 1.001
                    sl = buying_price * 0.996
                else:
                    print(f"Error in buying the order {buy_order}")
            else:
                sell_order = place_order('S')
                
                ltp = float(redis_client.hget('stock_data', TOKEN))
                res = client.order_report()
                selling_price = float(res['data'][-1]['avgPrc'])
                if sell_order and sell_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Sold at {selling_price}")
                    # order_count += 1
                    inPosition = True
                    tar = selling_price * 0.999
                    sl = selling_price * 1.003
                else:
                    print(f"Error in selling the order {sell_order} , Ltp is : {ltp}")
        
        if inPosition:
            ltp = float(redis_client.hget('stock_data', TOKEN))
            if pnl == 1:  # Currently holding a buy position
                if ltp >= tar:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Profit!")
                        pnl = 1
                        order_count += 1
                        inPosition = False
                        # time.sleep(5)
                elif ltp <= sl:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Loss!")
                        pnl = 0
                        order_count += 1
                        inPosition = False
            else:  # Currently holding a sell position
                if ltp <= tar:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Profit!")
                        pnl = 0
                        inPosition = False
                        # time.sleep(5)
                elif ltp >= sl:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Loss!")
                        pnl = 1
                        inPosition = False

            time.sleep(0.25)  # Check every 1 second
    else:
        print("Market is closed.")
        break

print("Order reached order_count.")

In [ ]:
# Main trading logic
order_count = 0
pnl = 1  # Start with profit state
inPosition = False
buying_price = 0
selling_price = 0

# pnl = find_pos()

while order_count < 100:
    current_time = datetime.datetime.now().time()
    if current_time >= datetime.time(9, 0) and current_time <= datetime.time(15,0):
        if not inPosition:
            # the strategy is starts from here
            if pnl == 1:
                buy_order = place_order('B')
                res = client.order_report()
                ltp = float(redis_client.hget('stock_data', TOKEN))
                buying_price = float(res['data'][-1]['avgPrc'])
                
                if buy_order and buy_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Bought at {buying_price} , Ltp is : {ltp}")
                    
                    inPosition = True
                    tar = buying_price * 1.001
                    sl = buying_price * 0.996
                else:
                    print(f"Error in buying the order {buy_order}")
            else:
                sell_order = place_order('S')
                
                ltp = float(redis_client.hget('stock_data', TOKEN))
                res = client.order_report()
                selling_price = float(res['data'][-1]['avgPrc'])
                if sell_order and sell_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Sold at {selling_price}")
                    # order_count += 1
                    inPosition = True
                    tar = selling_price * 0.999
                    sl = selling_price * 1.003
                else:
                    print(f"Error in selling the order {sell_order} , Ltp is : {ltp}")
        
        if inPosition:
            ltp = float(redis_client.hget('stock_data', TOKEN))
            if pnl == 1:  # Currently holding a buy position
                if ltp >= tar:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Profit!")
                        pnl = 1
                        order_count += 1
                        inPosition = False
                        # time.sleep(5)
                elif ltp <= sl:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Loss!")
                        pnl = 0
                        order_count += 1
                        inPosition = False
            else:  # Currently holding a sell position
                if ltp <= tar:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Profit!")
                        pnl = 0
                        inPosition = False
                        # time.sleep(5)
                elif ltp >= sl:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Loss!")
                        pnl = 1
                        inPosition = False

            time.sleep(0.25)  # Check every 1 second
    else:
        print("Market is closed.")
        break

print("Order reached order_count.")

In [ ]:
# Function to place order
def place_order(transaction_type):
    try:
        # Place an order
        # ltp = float(redis_client.hget('stock_data', TOKEN))
        # quant = str(int((TOTAL_FUND/ltp)*5) - 2)
        
        order = client.place_order(
            exchange_segment="nse_cm",          # Exchange segment for NSE cash market
            product="MIS",                      # Product type for cash and carry
            price="0",                          # Leave price empty for market order
            order_type="MKT",                  # Order type for market order
            quantity=1,                      # Quantity of the order (adjust as needed)
            validity="DAY",                    # Validity of the order
            trading_symbol=TRADING_SYMBOL,         # Trading symbol for Tata Steel
            transaction_type=transaction_type,              # Transaction type for buying
            amo="NO",                          # After market order (AMO) - NO for regular trading hours
            disclosed_quantity="0",            # Disclosed quantity
            market_protection="0",             # Market protection
            pf="N",                            # Protect flag (N for no protection)
            trigger_price="0",                 # Trigger price (not needed for market orders)
            tag=None                           # Optional tag for the order
        )
        # print("f{} Order placed successfully:",transaction_type, response)
        return order
    except Exception as e:
        print("Exception when calling OrderApi->place_order: %s\n" % e)


In [ ]:
def place_order(transaction_type, price=None):
    try:
        order = client.place_order(
            exchange_segment=EXCHANGE_SEGMENT,
            product=PRODUCT,
            price=str(price) if price else None,
            order_type=ORDER_TYPE,
            quantity=QUANTITY,
            validity=VALIDITY,
            trading_symbol=TRADING_SYMBOL,
            transaction_type=transaction_type,
            amo='NO',
            disclosed_quantity='0',
            market_protection='0',
            pf='N',
            trigger_price='0',
            tag=None
        )
        return order
    except Exception as e:
        print(f"Exception when calling OrderApi->place_order: {e}")
        return None

In [ ]:
# code from sockets :

# Test to retrieve and print the stored data
stored_data = redis_client.hgetall('stock_data')
print("Stored Data in Redis:")
for token, ltp in stored_data.items():
    print(f"Token: {token.decode('utf-8')}, LTP: {ltp.decode('utf-8')}")

# quote_type can be market_depth, ohlc, ltp, 52w, circuit_limits, scrip_details
# By Default quote_type is set as None that means you will get the complete data.
# Quotes api can be accessed without completing login by passing session_token, sid and server_id 


# print(type(res))
# print(res)
def on_message(message):
    print('[Res]: ', message)

def on_error(message):
    result = message
    print('[OnError]: ', result)
    
def on_open(message):
    print('[OnOpen]: ', message)
    
def on_close(message):
    print('[OnClose]: ', message)

# Setup Callbacks for websocket events (Optional)
client.on_message = on_message  # called when message is received from websocket
client.on_error = on_error  # called when any error or exception occurs in code or websocket
client.on_close = on_close  # called when websocket connection is closed
client.on_open = on_open  # called when websocket successfully connects

# try:
#     # Get live feed data
#     res = client.subscribe(instrument_tokens=instrument_tokens)
#     print(res)
# except Exception as e:
#     print("Exception while connection to socket->socket: %s\n  " % e)



# old code :


# # functions to store values on redis 

# # Redis connection
# redis_client = redis.Redis(host='localhost', port=6379, db=0)



# def store_in_redis(data):
#     print("in the store_in_redis function")
#     for item in data:
#         token = item.get('tk')
#         ltp = item.get('ltp')
#         if token and ltp:
#             redis_client.hset('stock_data', token, ltp)

# def process_responses(responses):
#     print("in the process_responses function")
#     for response in responses:
#         if response['type'] == 'stock_feed':
#             print(response)
#             # store_in_redis(response['data'])


# import redis

# redis_client = redis.Redis(host='localhost', port=6379, db=0)
# responses = [
#     {
#         'type': 'stock_feed',
#         'data': [
#             # {'tk': '383', 'ltp': '312.45', 'other_field': 'value'},
#             {'tk': '1624', 'ltp': '168.20', 'other_field': 'value'},
#             # Other responses...
#         ]
#     },
#     {
#         'type': 'stock_feed',
#         'data': [
#             # {'tk': '383', 'ltp': '312.50', 'other_field': 'value'},
#             {'tk': '21951', 'ltp': '117.37', 'other_field': 'value'},
#             # Other responses...
#         ]
#     }
#     # Additional responses...
# ]

# def store_in_redis(data):
#     for item in data:
#         token = item.get('tk')
#         ltp = item.get('ltp')
#         if token and ltp:
#             redis_client.hset('stock_data', token, ltp)

# def process_responses(responses):
#     for response in responses:
#         if response['type'] == 'stock_feed':
#             store_in_redis(response['data'])



# process_responses(responses)



def startWebsocket():
    client.subscribe(instrument_tokens=[{"instrument_token": "3426", "exchange_segment": "nse_cm"}],isIndex=False,isDepth=False)


startWebsocket()





# message =   {'type': 'stock_feed', 'data': [{'ftm0': '02/01/1970 19:24:48', 'dtm1': '30/09/1971 21:40:50', 'ltt': '22/07/2024 15:18:14', 'v': '16189851', 'ltp': '168.16', 'tbq': '2054849', 'tsq': '3844813', 'bp': '168.16', 'bq': '768', 'bs': '10408', 'cng': '2.67', 'nc': '1.61', 'to': '2709371564.85', 'name': 'sf', 'tk': '1624', 'e': 'nse_cm', 'request_type': 'SUB'}, {'ftm0': '02/01/1970 10:38:14', 'dtm1': '30/09/1971 21:40:50', 'ltt': '22/07/2024 15:18:14', 'v': '14263830', 'ltq': '51', 'tbq': '1305748', 'tsq': '2095594', 'sp': '117.45', 'bq': '727', 'bs': '186', 'to': '1652749982.10', 'name': 'sf', 'tk': '21951', 'e': 'nse_cm', 'request_type': 'SUB'}, {'ftm0': '02/01/1970 18:43:39', 'dtm1': '30/09/1971 21:40:50', 'name': 'sf', 'tk': '2031', 'e': 'nse_cm', 'request_type': 'SUB'}, {'ftm0': '02/01/1970 20:14:42', 'dtm1': '30/09/1971 21:40:50', 'fdtm': '22/07/2024 15:18:14', 'ltt': '22/07/2024 15:18:14', 'v': '8896554', 'ltp': '424.65', 'ltq': '3', 'tbq': '760182', 'tsq': '1049831', 'sp': '424.75', 'bq': '344', 'bs': '288', 'cng': '10.50', 'nc': '2.54', 'to': '3732905092.86', 'name': 'sf', 'tk': '3426', 'e': 'nse_cm', 'request_type': 'SUB'}]}

message =  {'type': 'stock_feed', 'data': [{'ftm0': '02/01/1970 01:59:09', 'dtm1': '31/05/1971 16:23:26', 'fdtm': '23/07/2024 12:22:15', 'ltt': '23/07/2024 12:22:14', 'v': '25611859', 'ltp': '300.55', 'ltq': '18', 'tbq': '916629', 'tsq': '5456672', 'bp': '300.50', 'sp': '300.65', 'bq': '703', 'bs': '211', 'ap': '310.85', 'lo': '298.00', 'h': '317.00', 'lcl': '281.10', 'ucl': '343.50', 'yh': '340.50', 'yl': '123.50', 'op': '317.00', 'c': '312.30', 'mul': '1', 'prec': '2', 'cng': '-11.75', 'nc': '-3.76', 'to': '7961446370.15', 'name': 'sf', 'tk': '383', 'e': 'nse_cm', 'ts': 'BEL-EQ', 'request_type': 'SNAP'}, {'ftm0': '02/01/1970 01:33:02', 'dtm1': '31/05/1971 16:23:26', 'fdtm': '23/07/2024 12:22:14', 'ltt': '23/07/2024 12:22:14', 'v': '8824863', 'ltp': '163.67', 'ltq': '2', 'tbq': '1221690', 'tsq': '5352377', 'bp': '163.65', 'sp': '163.67', 'bq': '1019', 'bs': '198', 'ap': '166.55', 'lo': '163.65', 'h': '169.35', 'lcl': '151.37', 'ucl': '185.00', 'yh': '196.80', 'yl': '85.50', 'op': '169.35', 'c': '168.19', 'mul': '1', 'prec': '2', 'cng': '-4.52', 'nc': '-2.69', 'to': '1469780932.65', 'name': 'sf', 'tk': '1624', 'e': 'nse_cm', 'ts': 'IOC-EQ', 'request_type': 'SNAP'}, {'ftm0': '01/01/1970 20:23:59', 'dtm1': '31/05/1971 16:23:26', 'fdtm': '23/07/2024 12:22:14', 'ltt': '23/07/2024 12:22:14', 'v': '6815824', 'ltp': '112.82', 'ltq': '9', 'tbq': '627184', 'tsq': '3454456', 'bp': '112.62', 'sp': '112.71', 'bq': '50', 'bs': '312', 'ap': '115.86', 'lo': '112.20', 'h': '117.89', 'lcl': '93.96', 'ucl': '140.94', 'yh': '135.40', 'yl': '61.50', 'op': '117.50', 'c': '117.45', 'mul': '1', 'prec': '2', 'cng': '-4.63', 'nc': '-3.94', 'to': '789681368.64', 'name': 'sf', 'tk': '21951', 'e': 'nse_cm', 'ts': 'HFCL-EQ', 'request_type': 'SNAP'}, {'ftm0': '01/01/1970 23:29:54', 'dtm1': '31/05/1971 16:23:26', 'fdtm': '23/07/2024 12:22:14', 'ltt': '23/07/2024 12:22:14', 'v': '1285564', 'ltp': '2794.20', 'ltq': '39', 'tbq': '65567', 'tsq': '254112', 'bp': '2793.05', 'sp': '2794.20', 'bq': '46', 'bs': '6', 'ap': '2824.77', 'lo': '2789.45', 'h': '2854.50', 'lcl': '2524.30', 'ucl': '3085.20', 'yh': '3013.50', 'yl': '1415.75', 'op': '2824.00', 'c': '2804.75', 'mul': '1', 'prec': '2', 'cng': '-10.55', 'nc': '-0.38', 'to': '3631422620.28', 'name': 'sf', 'tk': '2031', 'e': 'nse_cm', 'ts': 'M&M-EQ', 'request_type': 'SNAP'}, {'ftm0': '02/01/1970 01:41:17', 'dtm1': '31/05/1971 16:16:45', 'fdtm': '23/07/2024 12:22:14', 'ltt': '23/07/2024 12:22:14', 'v': '8579673', 'ltp': '410.10', 'ltq': '1', 'tbq': '437282', 'tsq': '1841339', 'bp': '410.00', 'sp': '410.10', 'bq': '9021', 'bs': '219', 'ap': '420.64', 'lo': '407.20', 'h': '428.65', 'lcl': '381.65', 'ucl': '466.45', 'yh': '464.20', 'yl': '216.80', 'op': '428.65', 'c': '424.05', 'mul': '1', 'prec': '2', 'cng': '-13.95', 'nc': '-3.29', 'to': '3608953650.72', 'name': 'sf', 'tk': '3426', 'e': 'nse_cm', 'ts': 'TATAPOWER-EQ', 'request_type': 'SNAP'}]
            
            }


for item in message['data']:
    token = item.get('tk')
    ltp = item.get('ltp')
    if token and ltp:
        # redis_client.hset('stock_data', token, ltp)
        print('token :', token)
        print('ltp :', ltp)




import threading
import time
# def startWebsocket():
#     client.subscribe(instrument_tokens=[{"instrument_token": "18943", "exchange_segment": "nse_cm"}],isIndex=False,isDepth=False)

threading.Thread(target=startWebsocket).start()
time.sleep(10)
threading.Thread(target=closeWebsocket).start()



# placing an order

try:
    # Place an order
    response = client.place_order(
        exchange_segment="nse_cm",          # Exchange segment for NSE cash market
        product="MIS",                      # Product type for cash and carry
        price="0",                          # Leave price empty for market order
        order_type="MKT",                  # Order type for market order
        quantity="1",                      # Quantity of the order (adjust as needed)
        validity="DAY",                    # Validity of the order
        trading_symbol="TATASTEEL-EQ",         # Trading symbol for Tata Steel
        transaction_type="B",              # Transaction type for buying
        amo="NO",                          # After market order (AMO) - NO for regular trading hours
        disclosed_quantity="0",            # Disclosed quantity
        market_protection="0",             # Market protection
        pf="N",                            # Protect flag (N for no protection)
        trigger_price="0",                 # Trigger price (not needed for market orders)
        tag=None                           # Optional tag for the order
    )
    print("Order placed successfully:", response)
except Exception as e:
    print("Exception when calling OrderApi->place_order: %s\n" % e)

In [ ]:
# old codde : bug ---> was putting 20 orders straight , even if position is already taken

# Main trading logic
order_count = 0
pnl = 1  # Start with profit state


while order_count < 20:
    current_time = datetime.datetime.now().time()
    if current_time >= datetime.time(9, 0) and current_time <= datetime.time(15, 15):
        if pnl == 1:
            buy_order = place_order('B')
            if buy_order and buy_order.get('stat') == 'Ok':
                buying_price = float(redis_client.hget('stock_data', TOKEN))
                print(f"Bought at {buying_price}")
                order_count += 1
                tar = buying_price * 1.001
                sl = buying_price * 0.998
                while True:
                    ltp = float(redis_client.hget('stock_data', TOKEN))
                    
                    if ltp >= tar:
                        sell_order = place_order('S')
                        if sell_order and sell_order.get('stat') == 'Ok':
                            print(f"Sold at {ltp}, Profit!")
                            pnl = 1
                        break
                    elif ltp <= sl:
                        sell_order = place_order('S')
                        if sell_order and sell_order.get('stat') == 'Ok':
                            print(f"Sold at {ltp}, Loss!")
                            pnl = 0
                        break
                    time.sleep(1)  # Check every 5 seconds

        elif pnl == 0:
            sell_order = place_order('S')
            if sell_order and sell_order.get('stat') == 'Ok':
                selling_price = float(redis_client.hget('stock_data', TOKEN))
                print(f"Sold at {selling_price}")
                order_count += 1
                tar = selling_price * 0.999
                sl = selling_price * 1.002
                
                while True:
                    ltp = float(redis_client.hget('stock_data', TOKEN))
                    if ltp <= tar:
                        buy_order = place_order('B')
                        if buy_order and buy_order.get('stat') == 'Ok':
                            print(f"Bought at {ltp}, Profit!")
                            pnl = 0
                        break
                    elif ltp >= sl:
                        buy_order = place_order('B')
                        if buy_order and buy_order.get('stat') == 'Ok':
                            print(f"Bought at {ltp}, Loss!")
                            pnl=1
                        break
                    time.sleep(1)  # Check every 5 seconds
    else:
        print("Market is closed.")
        break

print("Order reached order_count.")

In [ ]:
# very simple code to just check the data from the db and check conditions

# Main trading logic
order_count = 0
pnl = 1  # Start with profit state
inPosition = False
buying_price = 0
selling_price = 0

while order_count < 3:
    if not inPosition:
            if pnl == 1:
                buy_order = place_order('B')
                if buy_order and buy_order.get('stat') == 'Ok':
                    buying_price = float(redis_client.hget('stock_data', TOKEN))
                    print(f"Bought at {buying_price}")
                    order_count += 1
                    inPosition = True
                    tar = buying_price * 1.001
                    sl = buying_price * 0.998
            else:
                sell_order = place_order('S')
                if sell_order and sell_order.get('stat') == 'Ok':
                    selling_price = float(redis_client.hget('stock_data', TOKEN))
                    print(f"Sold at {selling_price}")
                    order_count += 1
                    inPosition = True
                    tar = selling_price * 0.999
                    sl = selling_price * 1.002
        
        while inPosition:
            ltp = float(redis_client.hget('stock_data', TOKEN))
            if pnl == 1:  # Currently holding a buy position
                if ltp >= tar:
                    sell_order = place_order('S')
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Profit!")
                        pnl = 1
                        inPosition = False
                elif ltp <= sl:
                    sell_order = place_order('S')
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Loss!")
                        pnl = 0
                        inPosition = False
            else:  # Currently holding a sell position
                if ltp <= tar:
                    buy_order = place_order('B')
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Profit!")
                        pnl = 0
                        inPosition = False
                elif ltp >= sl:
                    buy_order = place_order('B')
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Loss!")
                        pnl = 1
                        inPosition = False

            time.sleep(1)  # Check every 1 second


print("Order reached order_count.")


In [ ]:
# code from sockets

import time


print(time.ctime())

In [ ]:
# saved code of data on 18 oct , 1 : 45 am in the morning

import numpy as np
import datetime

# Function to calculate RSI (like TradingView)
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) # first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) # first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean()
    return 100 - 100 / (1 + rs)

# Function to calculate Stochastic RSI (like TradingView)
def StochRSI(series, period=14, smoothK=3, smoothD=3):
    rsi = RSI(series, period)
    rsi_min = rsi.rolling(window=period).min()
    rsi_max = rsi.rolling(window=period).max()
    stoch_rsi = 100 * (rsi - rsi_min) / (rsi_max - rsi_min)

    k = stoch_rsi.rolling(window=smoothK).mean()  # %K line
    d = k.rolling(window=smoothD).mean()  # %D line

    return k, d

def check_sell_signal(df):
    # Get the last two rows (current and previous) of %K and %D values
    last_row = df.iloc[-1]
    prev_row = df.iloc[-2]

    # Condition 1: K line crossing D line from above
    k_cross_d = prev_row['%K'] > prev_row['%D'] and last_row['%K'] < last_row['%D']
    # print(prev_row['%K'],prev_row['%D'],last_row['%K'],last_row['%D'])
    
    # Condition 2: Difference between the current and previous K values is more than 7
    k_difference = abs(last_row['%K'] - prev_row['%K']) > 2
    
    valid = prev_row['%K'] > 15 and prev_row['%K'] < 95

    # Generate a sell signal if both conditions are met
    if k_cross_d and k_difference and valid:
        return True
    else:
        return False
    

def get_data(date, symbol):
    start_time = pd.Timestamp(f'{date} 00:00:00')
    end_time = pd.Timestamp(f'{date} 23:59:59')
    data = yf.download(symbol, start=start_time, end=end_time, interval='5m')
    data = data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'})

    # Replace missing volume data with zero
    data['volume'] = data['volume'].fillna(0)

    data.index = data.index.tz_localize(None)
    # Reset index to 'start_Time'
    data.index.name = 'start_Time'

    # data = data.drop('Adj Close')
    # print(type(data))
    data  = data.drop(columns=['Adj Close'])
    return data

# Function to simulate signal generation
def generate_signal(stock_id, signal_type):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if signal_type == 'sell':
        signal_data = {
            'stock_id': stock_id,
            'signal': 'S',
            'time': current_time
            
            
        }
        # Push the signal to Redis (could use a list or stream)
        redis_conn.rpush('stock_signals', json.dumps(signal_data))

    else:
        signal_data = {
            'stock_id': stock_id,
            'signal': 'B',
            'time': current_time
            
        }
        # Push the signal to Redis (could use a list or stream)
        redis_conn.rpush('stock_signals', json.dumps(signal_data))
        


def aggregate_to_5min_df(data):
    # Resampling the data to 15-minute intervals
    data_5min = data.resample('5T').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    
    # Dropping any rows where the 'open' value is missing (incomplete 15 min intervals)
    data_5min = data_5min.dropna(subset=['open'])
    
    return data_5min

def get_dhan_data(token_code):
    data = dhan.intraday_minute_data(
        security_id=token_code,
        exchange_segment='NSE_EQ',
        instrument_type='EQUITY'
    )
    # print(data)
    data = data['data']
    # print(da)
    df1 = pd.DataFrame(data)
    
    # print(df1)
    # print(data)
    df1['start_Time'] = df1['start_Time'].apply(dhan.convert_to_date_time)
    df1.set_index('start_Time', inplace=True)
    df1=aggregate_to_5min_df(df1)
    return df1
    # print(df.head(152))
    # data = aggregate_to_15min(data)
    # print("----------------------------------------------------------")
    # df = pd.DataFrame(data)
    # df['start_Time'] = df['start_Time'].apply(dhan.convert_to_date_time)
    # # df['start_Time'] = df['start_Time'].apply(dhan.convert_to_date_time)
    



